In [110]:
from jsonschema import validate, ValidationError
from json import load

In [111]:
DIRECTORY = 'batch3'
# with open(path.join(DIRECTORY, 'cpb-aacip-15-7h1dj58h6r.json'), 'r') as f:
#     import json
#     json_data = json.load(f)

# json_data

In [112]:
from os import path
import json
SCHEMA='schemas/pbcore.schema.json'

with open(SCHEMA, 'r') as schema_file:
    schema = load(schema_file)

def validate_json(json_data, schema_file=schema):
    try:
        validate(instance=json_data, schema=schema)
        return True
    except ValidationError as e:
        return e.message




In [113]:
# validate_json(json_data)
# validate(json_data, schema)

In [114]:
import os
errors = []
for path, subdirs, files in os.walk(DIRECTORY):
    for name in files:
        if name.startswith('cpb-aacip-') and name.endswith('.json'):
            with open(os.path.join(path, name), 'r') as f:
                json_data = json.load(f)
                try:
                    validate(json_data, schema)
                    # print(f"{name} is valid.")
                except ValidationError as e:
                    print(f"{name} is invalid: {e.message}")
                    errors.append((name, e.message))


In [115]:
len(errors)

0